# Process CFReT single cell morphology features from CellProfiler readout

## Import libraries

In [1]:
import pathlib
import pandas as pd

from pycytominer import normalize, feature_select
from pycytominer.cyto_utils import cells, output

import sys
sys.path.append("../../utils")
import extraction_utils as extract_utils

## Set up paths

There are two plates from the CFReT data:

1) localhost220512140003_KK22-05-198 - contains images from well columns 1-8 (only wells that did not have phalloidin blow out)
2) localhost220513100001_KK22-05-198_FactinAdjusted - contains images from well columns 9-12 (wells that were adjusted for the phalloidin blow out)

These two plates images together make a full 384-well plate, but they are kept in their separate plates in case there are any batch effects.

In [2]:
# Set file and directory constants
cp_dir = "../2.cellprofiler_processing"
output_dir = "data"

# Set paths for plate localhost220512140003_KK22-05-198
sql_file1 = "localhost220512140003_KK22-05-198.sqlite"
single_cell_file1 = f"sqlite:///{cp_dir}/CellProfiler_output/{sql_file1}"
platemap_file1 = "metadata/plate_1_CFReT.csv"
sc_output_file1 = pathlib.Path(f"{output_dir}/localhost220512140003_KK22-05-198_sc_cellprofiler.csv.gz")
sc_norm_output_file1 = pathlib.Path(f"{output_dir}/localhost220512140003_KK22-05-198_sc_norm_cellprofiler.csv.gz")
sc_norm_fs_output_file1 = pathlib.Path(f"{output_dir}/localhost220512140003_KK22-05-198_sc_norm_fs_cellprofiler.csv.gz")

# Set paths for plate localhost220513100001_KK22-05-198_FactinAdjusted
sql_file2 = "localhost220513100001_KK22-05-198_FactinAdjusted.sqlite"
single_cell_file2 = f"sqlite:///{cp_dir}/CellProfiler_output/{sql_file2}"
platemap_file2 = "metadata/plate_2_CFReT.csv"
sc_output_file2 = pathlib.Path(f"{output_dir}/localhost220513100001_KK22-05-198_FactinAdjusted_sc_cellprofiler.csv.gz")
sc_norm_output_file2 = pathlib.Path(f"{output_dir}/localhost220513100001_KK22-05-198_FactinAdjusted_sc_norm_cellprofiler.csv.gz")
sc_norm_fs_output_file2 = pathlib.Path(f"{output_dir}/localhost220513100001_KK22-05-198_FactinAdjusted_sc_norm_fs_cellprofiler.csv.gz")

## Set up names for linking columns between tables in the database file

In [3]:
# Define custom linking columns between compartments
linking_cols = {
    "Per_Cytoplasm": {
        "Per_Cells": "Cytoplasm_Parent_Cells",
        "Per_Nuclei": "Cytoplasm_Parent_Nuclei",
    },
    "Per_Cells": {"Per_Cytoplasm": "Cells_Number_Object_Number"},
    "Per_Nuclei": {"Per_Cytoplasm": "Nuclei_Number_Object_Number"},
}

## Load and view platemaps file per plate

### Plate localhost220512140003_KK22-05-198

In [4]:
# Load platemap file for plate localhost220512140003_KK22-05-198
platemap_df1 = pd.read_csv(platemap_file1)
platemap_df1

print(platemap_df1.shape)
platemap_df1.head()

(64, 6)


,WellRow,WellCol,well_position,heart_number,treatment,dose
0,A,1,A01,3,drug_x,5uM
1,A,2,A02,3,drug_x,10uM
2,A,3,A03,3,drug_x,10uM
3,A,4,A04,3,DMSO,0uM
4,A,5,A05,8,drug_x,5uM


### Plate localhost220513100001_KK22-05-198_FactinAdjusted

In [5]:
# Load platemap file for plate localhost220513100001_KK22-05-198_FactinAdjusted
platemap_df2 = pd.read_csv(platemap_file2)
platemap_df2

print(platemap_df2.shape)
platemap_df2.head()

(32, 6)


,WellRow,WellCol,well_position,heart_number,treatment,dose
0,A,9,A09,9,drug_x,5uM
1,A,10,A10,9,drug_x,10uM
2,A,11,A11,9,drug_x,10uM
3,A,12,A12,9,DMSO,0uM
4,B,9,B09,9,drug_x,5uM


## Set up `SingleCells` class from Pycytominer

### Plate localhost220512140003_KK22-05-198

In [6]:
# Instantiate SingleCells class
sc1 = cells.SingleCells(
    sql_file=single_cell_file1,
    compartments=["Per_Cells", "Per_Cytoplasm", "Per_Nuclei"],
    compartment_linking_cols=linking_cols,
    image_table_name="Per_Image",
    strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
    merge_cols=["ImageNumber"],
    image_cols="ImageNumber",
    load_image_data=True
)

/home/jenna/anaconda3/envs/3.process-cfret-features/lib/python3.8/site-packages/pycytominer/cyto_utils/util.py:61: UserWarning: Non-canonical compartment detected: per_cells, per_cytoplasm, per_nuclei
  warnings.warn(warn_str)


### Plate localhost220513100001_KK22-05-198_FactinAdjusted

In [7]:
# Instantiate SingleCells class
sc2 = cells.SingleCells(
    sql_file=single_cell_file2,
    compartments=["Per_Cells", "Per_Cytoplasm", "Per_Nuclei"],
    compartment_linking_cols=linking_cols,
    image_table_name="Per_Image",
    strata=["Image_Metadata_Well", "Image_Metadata_Plate"],
    merge_cols=["ImageNumber"],
    image_cols="ImageNumber",
    load_image_data=True
)

/home/jenna/anaconda3/envs/3.process-cfret-features/lib/python3.8/site-packages/pycytominer/cyto_utils/util.py:61: UserWarning: Non-canonical compartment detected: per_cells, per_cytoplasm, per_nuclei
  warnings.warn(warn_str)


## Merge single cells

### Plate localhost220512140003_KK22-05-198

In [8]:
# Merge single cells across compartments
anno_kwargs1 = {"join_on": ["Metadata_well_position", "Image_Metadata_Well"]}

sc_df1 = sc1.merge_single_cells(
    platemap=platemap_df1,
    **anno_kwargs1,
)

# Save level 2 data as a csv
output(sc_df1, sc_output_file1)

print(sc_df1.shape)
sc_df1.head()

(41235, 2019)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Golgi_3_02_256,Nuclei_Texture_Variance_Golgi_3_03_256,Nuclei_Texture_Variance_Hoechst_3_00_256,Nuclei_Texture_Variance_Hoechst_3_01_256,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256
0,A,1,3,drug_x,5uM,1,localhost220512140003,A01,1,4,...,3.5556,3.496794,47.358214,46.360758,46.037829,45.900805,5.429009,5.041537,4.614009,4.538214
1,A,1,3,drug_x,5uM,1,localhost220512140003,A01,2,5,...,5.703998,5.618282,40.180746,41.678283,40.273453,41.117223,1.820114,1.964851,1.778454,1.796643
2,A,1,3,drug_x,5uM,1,localhost220512140003,A01,3,6,...,8.01034,7.534007,86.146958,86.876883,86.48231,86.468516,4.648866,4.38639,5.012021,4.646987
3,A,1,3,drug_x,5uM,1,localhost220512140003,A01,4,7,...,6.542603,6.886314,27.74657,26.991794,27.23615,25.985893,3.508829,3.042764,3.162068,3.174969
4,A,1,3,drug_x,5uM,1,localhost220512140003,A01,5,8,...,2.461162,2.464008,40.36296,38.84844,38.528401,38.782049,5.996352,5.167944,5.492333,5.74792


### Plate localhost220513100001_KK22-05-198_FactinAdjusted

In [9]:
# Merge single cells across compartments
anno_kwargs2 = {"join_on": ["Metadata_well_position", "Image_Metadata_Well"]}

sc_df2 = sc2.merge_single_cells(
    platemap=platemap_df2,
    **anno_kwargs2,
)

# Save level 2 data as a csv
output(sc_df2, sc_output_file2)

print(sc_df2.shape)
sc_df2.head()

(17352, 2019)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Golgi_3_02_256,Nuclei_Texture_Variance_Golgi_3_03_256,Nuclei_Texture_Variance_Hoechst_3_00_256,Nuclei_Texture_Variance_Hoechst_3_01_256,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256
0,A,9,9,drug_x,5uM,1,localhost220513100001,A09,1,3,...,40.940905,41.761362,16.891392,17.106739,16.837301,17.020937,65.249381,56.422768,56.547485,58.700026
1,A,9,9,drug_x,5uM,1,localhost220513100001,A09,2,4,...,4.494573,4.280534,26.470634,25.983759,26.63347,26.692085,2.387172,2.55088,2.485428,2.265039
2,A,9,9,drug_x,5uM,1,localhost220513100001,A09,3,7,...,21.462864,21.98625,141.944618,144.416879,153.297639,147.088477,22.590302,21.109874,23.395531,23.982229
3,A,9,9,drug_x,5uM,1,localhost220513100001,A09,4,8,...,84.689343,68.808177,58.920732,58.970008,62.194088,59.808636,134.447371,141.676296,170.853277,132.043724
4,A,9,9,drug_x,5uM,1,localhost220513100001,A09,5,9,...,8.037115,7.562004,15.877515,16.348438,16.337812,15.96957,6.821389,6.572461,6.699607,6.774949


## Normalize data

### Plate localhost220512140003_KK22-05-198

In [10]:
# Normalize single cell data and write to file
normalize_sc_df1 = normalize(
    sc_df1,
    method="standardize"
)

output(normalize_sc_df1, sc_norm_output_file1)

print(normalize_sc_df1.shape)
normalize_sc_df1.head()

(41235, 2019)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Golgi_3_02_256,Nuclei_Texture_Variance_Golgi_3_03_256,Nuclei_Texture_Variance_Hoechst_3_00_256,Nuclei_Texture_Variance_Hoechst_3_01_256,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256
0,A,1,3,drug_x,5uM,1,localhost220512140003,A01,1,4,...,-0.252754,-0.251763,-0.214510,-0.218254,-0.221128,-0.219833,-0.373502,-0.371307,-0.376427,-0.370919
1,A,1,3,drug_x,5uM,1,localhost220512140003,A01,2,5,...,-0.238055,-0.237122,-0.247561,-0.239617,-0.247667,-0.241613,-0.384977,-0.381218,-0.385427,-0.379697
2,A,1,3,drug_x,5uM,1,localhost220512140003,A01,3,6,...,-0.222276,-0.223900,-0.035896,-0.033402,-0.034924,-0.035126,-0.375983,-0.373418,-0.375163,-0.370571
3,A,1,3,drug_x,5uM,1,localhost220512140003,A01,4,7,...,-0.232318,-0.228370,-0.304818,-0.306623,-0.307690,-0.310507,-0.379607,-0.377746,-0.381036,-0.375284
4,A,1,3,drug_x,5uM,1,localhost220512140003,A01,5,8,...,-0.260242,-0.258891,-0.246722,-0.252528,-0.255701,-0.252245,-0.371699,-0.370900,-0.373639,-0.367046


### Plate localhost220513100001_KK22-05-198_FactinAdjusted

In [11]:
# Normalize single cell data and write to file
normalize_sc_df2 = normalize(
    sc_df2,
    method="standardize"
)

output(normalize_sc_df2, sc_norm_output_file2)

print(normalize_sc_df2.shape)
normalize_sc_df2.head()

(17352, 2019)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_Variance_Golgi_3_02_256,Nuclei_Texture_Variance_Golgi_3_03_256,Nuclei_Texture_Variance_Hoechst_3_00_256,Nuclei_Texture_Variance_Hoechst_3_01_256,Nuclei_Texture_Variance_Hoechst_3_02_256,Nuclei_Texture_Variance_Hoechst_3_03_256,Nuclei_Texture_Variance_Mitochondria_3_00_256,Nuclei_Texture_Variance_Mitochondria_3_01_256,Nuclei_Texture_Variance_Mitochondria_3_02_256,Nuclei_Texture_Variance_Mitochondria_3_03_256
0,A,9,9,drug_x,5uM,1,localhost220513100001,A09,1,3,...,0.190496,0.207431,-0.365832,-0.363089,-0.366385,-0.363583,0.182652,0.135270,0.130012,0.148471
1,A,9,9,drug_x,5uM,1,localhost220513100001,A09,2,4,...,-0.211100,-0.212366,-0.337573,-0.336976,-0.337459,-0.335111,-0.197739,-0.195503,-0.196986,-0.196385
2,A,9,9,drug_x,5uM,1,localhost220513100001,A09,3,7,...,-0.024129,-0.014056,0.003085,0.011414,0.036555,0.019334,-0.075486,-0.081551,-0.070510,-0.063678
3,A,9,9,drug_x,5uM,1,localhost220513100001,A09,4,8,...,0.672552,0.510364,-0.241842,-0.239941,-0.232456,-0.237616,0.601382,0.658725,0.821397,0.596652
4,A,9,9,drug_x,5uM,1,localhost220513100001,A09,5,9,...,-0.172065,-0.175612,-0.368823,-0.365320,-0.367860,-0.366678,-0.170907,-0.170810,-0.171496,-0.168827


## Feature selection

### Plate localhost220512140003_KK22-05-198

In [12]:
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

feature_select_norm_sc_df1 = feature_select(
    normalize_sc_df1,
    operation=feature_select_ops
)

output(feature_select_norm_sc_df1, sc_norm_fs_output_file1)

print(feature_select_norm_sc_df1.shape)
feature_select_norm_sc_df1.head()

(41235, 728)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_01_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_02_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_ER_3_01_256,Nuclei_Texture_SumEntropy_Golgi_3_03_256,Nuclei_Texture_SumVariance_Actin_3_03_256,Nuclei_Texture_SumVariance_ER_3_01_256,Nuclei_Texture_SumVariance_Golgi_3_03_256,Nuclei_Texture_SumVariance_Hoechst_3_03_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256
0,A,1,3,drug_x,5uM,1,localhost220512140003,A01,1,4,...,2.579048,1.920918,2.040439,-1.042963,-1.670381,-0.323247,-0.330685,-0.240433,-0.239336,-0.348498
1,A,1,3,drug_x,5uM,1,localhost220512140003,A01,2,5,...,3.093239,2.496232,2.811327,-0.815590,-1.246693,-0.405657,-0.331082,-0.228196,-0.248682,-0.357368
2,A,1,3,drug_x,5uM,1,localhost220512140003,A01,3,6,...,2.414257,2.430956,2.499516,-0.066393,-0.704874,-0.408283,-0.191011,-0.215951,-0.070878,-0.347841
3,A,1,3,drug_x,5uM,1,localhost220512140003,A01,4,7,...,2.525031,2.224401,2.303325,-0.714888,-0.737229,-0.411290,-0.323233,-0.212099,-0.294583,-0.353614
4,A,1,3,drug_x,5uM,1,localhost220512140003,A01,5,8,...,1.886933,1.707670,2.306141,-1.058874,-1.822871,-0.241135,-0.329911,-0.243944,-0.253741,-0.343661


### Plate localhost220513100001_KK22-05-198_FactinAdjusted

In [13]:
feature_select_ops = [
    "variance_threshold",
    "correlation_threshold",
    "blocklist",
]

feature_select_norm_sc_df2 = feature_select(
    normalize_sc_df2,
    operation=feature_select_ops
)

output(feature_select_norm_sc_df2, sc_norm_fs_output_file2)

print(feature_select_norm_sc_df2.shape)
feature_select_norm_sc_df2.head()

(17352, 628)


,Metadata_WellRow,Metadata_WellCol,Metadata_heart_number,Metadata_treatment,Metadata_dose,Metadata_ImageNumber,Metadata_Plate,Metadata_Well,Metadata_Cytoplasm_Parent_Cells,Metadata_Cytoplasm_Parent_Nuclei,...,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_01_256,Nuclei_Texture_InverseDifferenceMoment_Mitochondria_3_03_256,Nuclei_Texture_SumEntropy_ER_3_03_256,Nuclei_Texture_SumEntropy_Golgi_3_01_256,Nuclei_Texture_SumEntropy_Mitochondria_3_03_256,Nuclei_Texture_SumVariance_Actin_3_01_256,Nuclei_Texture_SumVariance_ER_3_03_256,Nuclei_Texture_SumVariance_Golgi_3_01_256,Nuclei_Texture_SumVariance_Hoechst_3_01_256,Nuclei_Texture_SumVariance_Mitochondria_3_03_256
0,A,9,9,drug_x,5uM,1,localhost220513100001,A09,1,3,...,-1.341714,-0.771456,0.383014,1.527986,1.603349,-0.420076,-0.236242,0.203806,-0.354730,0.161636
1,A,9,9,drug_x,5uM,1,localhost220513100001,A09,2,4,...,1.211139,0.965459,-0.922346,-0.491535,-1.000420,-0.460815,-0.468439,-0.206325,-0.331033,-0.189029
2,A,9,9,drug_x,5uM,1,localhost220513100001,A09,3,7,...,-1.117919,-1.411945,0.123190,0.934086,0.958949,-0.468958,-0.237587,-0.036968,-0.107819,-0.063200
3,A,9,9,drug_x,5uM,1,localhost220513100001,A09,4,8,...,-1.161359,-1.835825,0.908000,1.779381,1.798252,-0.463245,0.843474,0.579924,-0.257025,0.535400
4,A,9,9,drug_x,5uM,1,localhost220513100001,A09,5,9,...,0.059975,0.001208,-0.555080,0.129196,-0.043079,-0.472035,-0.441913,-0.169445,-0.353311,-0.162409


## Add single cell counts for all `csv.gz` files

In [14]:
# Plate localhost220512140003_KK22-05-198
extract_utils.add_sc_count_metadata(sc_output_file1)
extract_utils.add_sc_count_metadata(sc_norm_output_file1)
extract_utils.add_sc_count_metadata(sc_norm_fs_output_file1)

# Plate localhost220513100001_KK22-05-198_FactinAdjusted
extract_utils.add_sc_count_metadata(sc_output_file2)
extract_utils.add_sc_count_metadata(sc_norm_output_file2)
extract_utils.add_sc_count_metadata(sc_norm_fs_output_file2)

---

### Visualize basic count statistics

### Plate localhost220512140003_KK22-05-198

In [15]:
sc_df1.Metadata_dose.value_counts()

0uM        10349
5uM        10308
0.04uM      2832
3.33uM      2751
1.11uM      2725
0.014uM     2682
0.12uM      2548
0.37uM      2485
0.005uM     2293
10uM        2262
Name: Metadata_dose, dtype: int64

In [16]:
pd.crosstab(sc_df1.Metadata_dose, sc_df1.Metadata_Well)

Metadata_Well,A01,A02,A03,A04,A05,A06,A07,A08,B01,B02,...,G07,G08,H01,H02,H03,H04,H05,H06,H07,H08
Metadata_dose,,,,,,,,,,,,,,,,,,,,,
0.005uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,682,537,0,0,542,532,0
0.014uM,0,0,0,0,0,0,0,0,0,0,...,763,0,0,0,0,0,0,0,0,0
0.04uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.12uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.37uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0uM,0,0,0,522,0,0,0,500,0,0,...,0,774,0,0,0,659,0,0,0,585
1.11uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10uM,0,652,546,0,0,538,526,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.33uM,0,0,0,0,0,0,0,0,0,627,...,0,0,0,0,0,0,0,0,0,0


### Plate localhost220513100001_KK22-05-198_FactinAdjusted

In [17]:
sc_df2.Metadata_dose.value_counts()

5uM        4711
0uM        3723
0.014uM    1506
0.04uM     1474
0.12uM     1464
0.37uM     1213
0.005uM    1021
1.11uM      785
3.33uM      773
10uM        682
Name: Metadata_dose, dtype: int64

In [18]:
pd.crosstab(sc_df2.Metadata_dose, sc_df2.Metadata_Well)

Metadata_Well,A09,A10,A11,A12,B09,B10,B11,B12,C09,C10,...,F11,F12,G09,G10,G11,G12,H09,H10,H11,H12
Metadata_dose,,,,,,,,,,,,,,,,,,,,,
0.005uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,605,416,0
0.014uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,840,666,0,0,0,0,0
0.04uM,0,0,0,0,0,0,0,0,0,0,...,680,0,0,0,0,0,0,0,0,0
0.12uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0.37uM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0uM,0,0,0,248,0,0,0,344,0,0,...,0,550,0,0,0,626,0,0,0,472
1.11uM,0,0,0,0,0,0,0,0,0,411,...,0,0,0,0,0,0,0,0,0,0
10uM,0,349,333,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3.33uM,0,0,0,0,0,371,402,0,0,0,...,0,0,0,0,0,0,0,0,0,0
